### Training a RL agent on Atari games

In [7]:
make_atari_env??

### CartPole Environment

### Multiprocessing: Unleashing the Power of Vectorized Environments

In [ ]:
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

def make_env(env_id: str, rank: int, seed: int = 0):
    """
    Utility function for multiprocessed env.

    :param env_id: the environment ID
    :param num_env: the number of environments you wish to have in subprocesses
    :param seed: the inital seed for RNG
    :param rank: index of the subprocess
    """
    def _init():
        env = gym.make(env_id, render_mode="human")
        env.reset(seed=seed + rank)
        return env
    set_random_seed(seed)
    return _init

# if __name__ == "__main__":

env_id = "CartPole-v1"
num_cpu = 4  # Number of processes to use
# Create the vectorized environment
vec_env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

# Stable Baselines provides you with make_vec_env() helper
# which does exactly the previous steps for you.
# You can choose between `DummyVecEnv` (usually faster) and `SubprocVecEnv`
# env = make_vec_env(env_id, n_envs=num_cpu, seed=0, vec_env_cls=SubprocVecEnv)

model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=50_000)
model.save("models/CartPole")

obs = vec_env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render()

Using cuda device


### Callbacks: Evaluate Agent Performance

In [ ]:
import os
import gymnasium as gym

from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env

env_id = "Pendulum-v1"
n_training_envs = 1
n_eval_envs = 5

# Create log dir where evaluation results will be saved
eval_log_dir = "./models/Pendulum_eval_logs/"
os.makedirs(eval_log_dir, exist_ok=True)

# Initialize a vectorized training environment with default parameters
train_env = make_vec_env(env_id, n_envs=n_training_envs, seed=0)

# Separate evaluation env, with different parameters passed via env_kwargs
# Eval environments can be vectorized to speed up evaluation.
eval_env = make_vec_env(env_id, n_envs=n_eval_envs, seed=0,
                        env_kwargs={'g':0.7})

# Create callback that evaluates agent for 5 episodes every 500 training environment steps.
# When using multiple training environments, agent will be evaluated every
# eval_freq calls to train_env.step(), thus it will be evaluated every
# (eval_freq * n_envs) training steps. See EvalCallback doc for more information.
eval_callback = EvalCallback(eval_env, best_model_save_path=eval_log_dir,
                              log_path=eval_log_dir, eval_freq=max(500 // n_training_envs, 1),
                              n_eval_episodes=5, deterministic=True,
                              render=False)

model = SAC("MlpPolicy", train_env)
# model = SAC.load("./models/Pendulum_eval_logs/best_model", env=train_env)

model.learn(50000, callback=eval_callback)


model = SAC.load("./models/Pendulum_eval_logs/best_model", env=train_env)

obs = train_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = train_env.step(action)
    train_env.render("human")

Eval num_timesteps=500, episode_reward=-131.90 +/- 121.17
Episode length: 200.00 +/- 0.00
New best mean reward!


#### Pong

In [ ]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C
import time

t0=time.time()

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
# Frame-stacking with 4 frames
vec_env = VecFrameStack(vec_env, n_stack=4)

# model = A2C("CnnPolicy", vec_env, verbose=1)
model = A2C.load("models/Pong", env=vec_env)

model.learn(total_timesteps=50_000)
model.save("models/Pong")

print(f"train time: {time.time()-t0}")

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

Wrapping the env in a VecTransposeImage.
------------------------------------
| time/                 |          |
|    fps                | 283      |
|    iterations         | 100      |
|    time_elapsed       | 7        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.03    |
|    explained_variance | 0.927    |
|    learning_rate      | 0.0007   |
|    n_updates          | 6349     |
|    policy_loss        | -0.00657 |
|    value_loss         | 0.00536  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 3.36e+03 |
|    ep_rew_mean        | -20.5    |
| time/                 |          |
|    fps                | 320      |
|    iterations         | 200      |
|    time_elapsed       | 12       |
|    total_timesteps    | 4000     |
| train/                |          |
|    entropy_loss       | -1.35    |
|    explained_variance | 0.303   

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 3.49e+03 |
|    ep_rew_mean        | -20.5    |
| time/                 |          |
|    fps                | 329      |
|    iterations         | 1500     |
|    time_elapsed       | 90       |
|    total_timesteps    | 30000    |
| train/                |          |
|    entropy_loss       | -1.5     |
|    explained_variance | 0.655    |
|    learning_rate      | 0.0007   |
|    n_updates          | 7749     |
|    policy_loss        | 0.0464   |
|    value_loss         | 0.0167   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 3.48e+03 |
|    ep_rew_mean        | -20.5    |
| time/                 |          |
|    fps                | 329      |
|    iterations         | 1600     |
|    time_elapsed       | 97       |
|    total_timesteps    | 32000    |
| train/                |          |
|

D:\apps\Anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


#### Breakout

In [ ]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C
import time

t0=time.time()

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
# vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=1, seed=0)
# Frame-stacking with 4 frames
vec_env = VecFrameStack(vec_env, n_stack=4)

model = A2C("CnnPolicy", vec_env, verbose=1)
# model = A2C.load("models/Breakout", env=vec_env)

model.learn(total_timesteps=20_000)
model.save("models/Breakout")

print(f"train time: {time.time()-t0}")

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")
    

Using cuda device
Wrapping the env in a VecTransposeImage.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 700      |
|    ep_rew_mean        | 1        |
| time/                 |          |
|    fps                | 85       |
|    iterations         | 100      |
|    time_elapsed       | 5        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.39    |
|    explained_variance | 0.222    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.000157 |
|    value_loss         | 3.21e-07 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 827      |
|    ep_rew_mean        | 1.8      |
| time/                 |          |
|    fps                | 102      |
|    iterations         | 200      |
|    time_elapsed       | 9        |
|    total_times

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 751      |
|    ep_rew_mean        | 1.4      |
| time/                 |          |
|    fps                | 122      |
|    iterations         | 1400     |
|    time_elapsed       | 57       |
|    total_timesteps    | 7000     |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | 0.12     |
|    learning_rate      | 0.0007   |
|    n_updates          | 1399     |
|    policy_loss        | -0.00099 |
|    value_loss         | 2.05e-06 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 748      |
|    ep_rew_mean        | 1.39     |
| time/                 |          |
|    fps                | 122      |
|    iterations         | 1500     |
|    time_elapsed       | 61       |
|    total_timesteps    | 7500     |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 735      |
|    ep_rew_mean        | 1.31     |
| time/                 |          |
|    fps                | 124      |
|    iterations         | 2700     |
|    time_elapsed       | 108      |
|    total_timesteps    | 13500    |
| train/                |          |
|    entropy_loss       | -0.193   |
|    explained_variance | -231     |
|    learning_rate      | 0.0007   |
|    n_updates          | 2699     |
|    policy_loss        | 0.00447  |
|    value_loss         | 9.51e-05 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 728       |
|    ep_rew_mean        | 1.26      |
| time/                 |           |
|    fps                | 124       |
|    iterations         | 2800      |
|    time_elapsed       | 112       |
|    total_timesteps    | 14000     |
| train/                |    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 770      |
|    ep_rew_mean        | 1.65     |
| time/                 |          |
|    fps                | 124      |
|    iterations         | 4000     |
|    time_elapsed       | 160      |
|    total_timesteps    | 20000    |
| train/                |          |
|    entropy_loss       | -0.196   |
|    explained_variance | -8.38    |
|    learning_rate      | 0.0007   |
|    n_updates          | 3999     |
|    policy_loss        | -0.00107 |
|    value_loss         | 0.00163  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 781      |
|    ep_rew_mean        | 1.74     |
| time/                 |          |
|    fps                | 124      |
|    iterations         | 4100     |
|    time_elapsed       | 164      |
|    total_timesteps    | 20500    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 820      |
|    ep_rew_mean        | 2.15     |
| time/                 |          |
|    fps                | 125      |
|    iterations         | 5300     |
|    time_elapsed       | 211      |
|    total_timesteps    | 26500    |
| train/                |          |
|    entropy_loss       | -0.195   |
|    explained_variance | -36.8    |
|    learning_rate      | 0.0007   |
|    n_updates          | 5299     |
|    policy_loss        | 0.000192 |
|    value_loss         | 0.000134 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 820      |
|    ep_rew_mean        | 2.15     |
| time/                 |          |
|    fps                | 125      |
|    iterations         | 5400     |
|    time_elapsed       | 215      |
|    total_timesteps    | 27000    |
| train/                |          |
|

...

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 770      |
|    ep_rew_mean        | 1.98     |
| time/                 |          |
|    fps                | 18       |
|    iterations         | 156100   |
|    time_elapsed       | 42431    |
|    total_timesteps    | 780500   |
| train/                |          |
|    entropy_loss       | -0.0384  |
|    explained_variance | 0.996    |
|    learning_rate      | 0.0007   |
|    n_updates          | 156099   |
|    policy_loss        | 3.95e-06 |
|    value_loss         | 9.85e-07 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 781       |
|    ep_rew_mean        | 2.07      |
| time/                 |           |
|    fps                | 18        |
|    iterations         | 156200    |
|    time_elapsed       | 42435     |
|    total_timesteps    | 781000    |
| train/                |    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 825      |
|    ep_rew_mean        | 2.43     |
| time/                 |          |
|    fps                | 18       |
|    iterations         | 157400   |
|    time_elapsed       | 42483    |
|    total_timesteps    | 787000   |
| train/                |          |
|    entropy_loss       | -0.0129  |
|    explained_variance | 0.996    |
|    learning_rate      | 0.0007   |
|    n_updates          | 157399   |
|    policy_loss        | 7.72e-06 |
|    value_loss         | 0.00066  |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 814       |
|    ep_rew_mean        | 2.34      |
| time/                 |           |
|    fps                | 18        |
|    iterations         | 157500    |
|    time_elapsed       | 42487     |
|    total_timesteps    | 787500    |
| train/                |    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 803       |
|    ep_rew_mean        | 2.25      |
| time/                 |           |
|    fps                | 18        |
|    iterations         | 158700    |
|    time_elapsed       | 42535     |
|    total_timesteps    | 793500    |
| train/                |           |
|    entropy_loss       | -0.00112  |
|    explained_variance | -46.7     |
|    learning_rate      | 0.0007    |
|    n_updates          | 158699    |
|    policy_loss        | -4.41e-10 |
|    value_loss         | 6e-10     |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 804      |
|    ep_rew_mean        | 2.25     |
| time/                 |          |
|    fps                | 18       |
|    iterations         | 158800   |
|    time_elapsed       | 42539    |
|    total_timesteps    | 794000   |
| train/             

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 826       |
|    ep_rew_mean        | 2.43      |
| time/                 |           |
|    fps                | 18        |
|    iterations         | 160000    |
|    time_elapsed       | 42586     |
|    total_timesteps    | 800000    |
| train/                |           |
|    entropy_loss       | -0.000879 |
|    explained_variance | 0.493     |
|    learning_rate      | 0.0007    |
|    n_updates          | 159999    |
|    policy_loss        | -2.24e-09 |
|    value_loss         | 1.03e-09  |
-------------------------------------
train time: 42587.294620513916


D:\apps\Anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


In [ ]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C

vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=1, seed=0)
# Frame-stacking with 4 frames
vec_env = VecFrameStack(vec_env, n_stack=4)

model = A2C.load("models/Breakout", env=vec_env)

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")
    

Wrapping the env in a VecTransposeImage.


D:\apps\Anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


In [12]:
A2C.load??

In [1]:
int(2e4)

20000